## 0 preparation

In [14]:
!pip install pip==23.3 # Need a downgrade or it reports 'Please use pip<24.1 if you need to use this version'.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [15]:
!git clone https://github.com/gitmylo/bark-voice-cloning-HuBERT-quantizer/
%cd bark-voice-cloning-HuBERT-quantizer

%pip install -r requirements.txt
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Cloning into 'bark-voice-cloning-HuBERT-quantizer'...
remote: Enumerating objects: 1882, done.
remote: Counting objects: 100% (247/247), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 1882 (delta 144), reused 215 (delta 124), pack-reused 1635 (from 1)
Receiving objects: 100% (1882/1882), 319.75 MiB | 25.29 MiB/s, done.
Resolving deltas: 100% (145/145), done.
/content/bark-voice-cloning-HuBERT-quantizer/bark-voice-cloning-HuBERT-quantizer/bark-voice-cloning-HuBERT-quantizer
Ignoring soundfile: markers 'platform_system == "Windows"' don't match your environment
  Using cached audiolm_pytorch-1.1.4-py3-none-any.whl.metadata (1.2 kB)
  Using cached fairseq-0.12.2.tar.gz (9.6 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached sox-1.5.0.tar.gz (63 kB)
  Preparing metadata (setup.py) ... done
  Using cached beartype-0.19.0-py

## 1 load models

In [17]:
large_quant_model = False  # Use the larger pretrained model
device = 'cp'  # 'cuda', 'cpu', 'cuda:0', 0, -1, torch.device('cuda')

import numpy as np
import torch
import torchaudio
from encodec import EncodecModel
from encodec.utils import convert_audio
from bark_hubert_quantizer.hubert_manager import HuBERTManager
from bark_hubert_quantizer.pre_kmeans_hubert import CustomHubert
from bark_hubert_quantizer.customtokenizer import CustomTokenizer

model = ('quantifier_V1_hubert_base_ls960_23.pth', 'tokenizer_large.pth') if large_quant_model else ('quantifier_hubert_base_ls960_14.pth', 'tokenizer.pth')

print('Loading HuBERT...')
hubert_model = CustomHubert(HuBERTManager.make_sure_hubert_installed(), device=device)
print('Loading Quantizer...')
quant_model = CustomTokenizer.load_from_checkpoint(HuBERTManager.make_sure_tokenizer_installed(model=model[0], local_file=model[1]), device)
print('Loading Encodec...')
encodec_model = EncodecModel.encodec_model_24khz()
encodec_model.set_target_bandwidth(6.0)
encodec_model.to(device)

print('Downloaded and loaded models!')

Loading HuBERT...


/usr/local/lib/python3.10/dist-packages/fairseq/checkpoint_utils.py:315: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(f, map_location=torch.device("cpu")

Loading Quantizer...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:834: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


quantifier_hubert_base_ls960_14.pth:   0%|          | 0.00/104M [00:00<?, ?B/s]

Downloaded tokenizer


/content/bark-voice-cloning-HuBERT-quantizer/bark-voice-cloning-HuBERT-quantizer/bark_hubert_quantizer/customtokenizer.py:119: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  

Loading Encodec...
Downloaded and loaded models!


## 2 load wav and create speaker history prompt

In [22]:
!pwd

/content/bark-voice-cloning-HuBERT-quantizer/bark-voice-cloning-HuBERT-quantizer/bark-voice-cloning-HuBERT-quantizer


In [27]:
from bark import SAMPLE_RATE, generate_audio, preload_models
from IPython.display import Audio

preload_models()

text_2.pt:   0%|          | 0.00/5.35G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/bark/generation.py:212: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location=device)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

coarse_2.pt:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

fine_2.pt:   0%|          | 0.00/3.74G [00:00<?, ?B/s]

In [28]:
text_prompt = """
    I have a silky smooth voice, and today I will tell you
    about the exercise regimen of the common sloth.
"""
audio_array = generate_audio(text_prompt, history_prompt="v2/en_speaker_1")
Audio(audio_array, rate=SAMPLE_RATE)

100%|██████████| 24/24 [08:45<00:00, 21.90s/it]


### 2.1 test 1: dog barking wav + english

In [26]:
wav_file = '/content/labrador-barking-daniel_simon.wav'  # Put the path of the speaker you want to use here.
out_file = '/content/labrador-barking-daniel_simon.npz'  # Put the path to save the cloned speaker to here.

wav, sr = torchaudio.load(wav_file)

wav_hubert = wav.to(device)

if wav_hubert.shape[0] == 2:  # Stereo to mono if needed
    wav_hubert = wav_hubert.mean(0, keepdim=True)

print('Extracting semantics...')
semantic_vectors = hubert_model.forward(wav_hubert, input_sample_hz=sr)
print('Tokenizing semantics...')
semantic_tokens = quant_model.get_token(semantic_vectors)
print('Creating coarse and fine prompts...')
wav = convert_audio(wav, sr, encodec_model.sample_rate, 1).unsqueeze(0)

wav = wav.to(device)

with torch.no_grad():
    encoded_frames = encodec_model.encode(wav)
codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1).squeeze()

codes = codes.cpu()
semantic_tokens = semantic_tokens.cpu()

np.savez(out_file,
         semantic_prompt=semantic_tokens,
         fine_prompt=codes,
         coarse_prompt=codes[:2, :]
         )

print('Done!')

Extracting semantics...
Tokenizing semantics...
Creating coarse and fine prompts...
Done!


In [32]:
model_path = "/content/labrador-barking-daniel_simon.npz"

text_prompt = """
    I have a silky smooth voice, and today I will tell you
    about the exercise regimen of the common sloth.
"""
audio_array = generate_audio(text_prompt, history_prompt=model_path)
Audio(audio_array, rate=SAMPLE_RATE)

100%|██████████| 26/26 [09:29<00:00, 21.91s/it]


### 2.2 test 2: chinese wav + english

In [33]:
wav_file = '/content/HuBERT_test_chinese.m4a'  # Put the path of the speaker you want to use here.
out_file = '/content/HuBERT_test_chinese.npz'  # Put the path to save the cloned speaker to here.

wav, sr = torchaudio.load(wav_file)

wav_hubert = wav.to(device)

if wav_hubert.shape[0] == 2:  # Stereo to mono if needed
    wav_hubert = wav_hubert.mean(0, keepdim=True)

print('Extracting semantics...')
semantic_vectors = hubert_model.forward(wav_hubert, input_sample_hz=sr)
print('Tokenizing semantics...')
semantic_tokens = quant_model.get_token(semantic_vectors)
print('Creating coarse and fine prompts...')
wav = convert_audio(wav, sr, encodec_model.sample_rate, 1).unsqueeze(0)

wav = wav.to(device)

with torch.no_grad():
    encoded_frames = encodec_model.encode(wav)
codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1).squeeze()

codes = codes.cpu()
semantic_tokens = semantic_tokens.cpu()

np.savez(out_file,
         semantic_prompt=semantic_tokens,
         fine_prompt=codes,
         coarse_prompt=codes[:2, :]
         )

print('Done!')

Extracting semantics...
Tokenizing semantics...
Creating coarse and fine prompts...
Done!


In [34]:
model_path = "/content/HuBERT_test_chinese.npz"

text_prompt = """
    I have a silky smooth voice, and today I will tell you
    about the exercise regimen of the common sloth.
"""
audio_array = generate_audio(text_prompt, history_prompt=model_path)
Audio(audio_array, rate=SAMPLE_RATE)

100%|██████████| 34/34 [12:24<00:00, 21.89s/it]


### 2.3 test 3: english wav + english

In [35]:
wav_file = '/content/HuBERT_test_english.m4a'  # Put the path of the speaker you want to use here.
out_file = '/content/HuBERT_test_english.npz'  # Put the path to save the cloned speaker to here.

wav, sr = torchaudio.load(wav_file)

wav_hubert = wav.to(device)

if wav_hubert.shape[0] == 2:  # Stereo to mono if needed
    wav_hubert = wav_hubert.mean(0, keepdim=True)

print('Extracting semantics...')
semantic_vectors = hubert_model.forward(wav_hubert, input_sample_hz=sr)
print('Tokenizing semantics...')
semantic_tokens = quant_model.get_token(semantic_vectors)
print('Creating coarse and fine prompts...')
wav = convert_audio(wav, sr, encodec_model.sample_rate, 1).unsqueeze(0)

wav = wav.to(device)

with torch.no_grad():
    encoded_frames = encodec_model.encode(wav)
codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1).squeeze()

codes = codes.cpu()
semantic_tokens = semantic_tokens.cpu()

np.savez(out_file,
         semantic_prompt=semantic_tokens,
         fine_prompt=codes,
         coarse_prompt=codes[:2, :]
         )

print('Done!')

Extracting semantics...
Tokenizing semantics...
Creating coarse and fine prompts...
Done!


In [36]:
model_path = "/content/HuBERT_test_english.npz"

text_prompt = """
    I have a silky smooth voice, and today I will tell you
    about the exercise regimen of the common sloth.
"""
audio_array = generate_audio(text_prompt, history_prompt=model_path)
Audio(audio_array, rate=SAMPLE_RATE)

100%|██████████| 27/27 [09:45<00:00, 21.68s/it]
